# Module 1: Financial Risk Metrics and Applications

This notebook demonstrates practical applications of probability theory in quantitative finance, focusing on risk measurement and portfolio analysis.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Calculate Value-at-Risk (VaR) using multiple methods
2. Compute Conditional VaR (Expected Shortfall)
3. Apply Monte Carlo simulation to option pricing
4. Calculate risk-adjusted performance metrics (Sharpe, Sortino)
5. Analyze real market data distributions
6. Model heavy-tailed distributions in finance

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, t as student_t
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Libraries imported successfully!")

## 1. Fetching Real Market Data

Let's start by downloading real stock price data and calculating returns.

In [ ]:
# Fetch historical data for major stocks
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'SPY']  # SPY is S&P 500 ETF
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"Downloading data from {start_date} to {end_date}...")
data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True, progress=False)

# Extract closing prices
prices = data['Close']
print(f"\nData shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
prices.head()

In [ ]:
# Calculate daily returns
returns = prices.pct_change().dropna()

# Display summary statistics
print("Summary Statistics of Daily Returns:\n")
print(returns.describe())

# Plot price evolution
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Normalized prices
normalized_prices = prices / prices.iloc[0]
normalized_prices.plot(ax=ax1, linewidth=2)
ax1.set_title('Normalized Stock Prices (Base = 100)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Normalized Price')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Returns distribution
returns.plot(ax=ax2, linewidth=1, alpha=0.7)
ax2.set_title('Daily Returns', fontsize=14, fontweight='bold')
ax2.set_ylabel('Returns')
ax2.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Value-at-Risk (VaR) Calculation

VaR answers the question: *"What is the maximum loss we can expect with X% confidence over a given time horizon?"*

We'll implement three methods:
1. **Historical VaR**: Using empirical distribution
2. **Parametric VaR**: Assuming normal distribution
3. **Cornish-Fisher VaR**: Adjusting for skewness and kurtosis

In [ ]:
def calculate_var(returns, confidence_level=0.95, method='historical'):
    """
    Calculate Value-at-Risk (VaR).
    
    Parameters:
    -----------
    returns : pd.Series or np.array
        Returns series
    confidence_level : float, default 0.95
        Confidence level (e.g., 0.95 for 95% VaR)
    method : str, default 'historical'
        Method: 'historical', 'parametric', or 'cornish-fisher'
    
    Returns:
    --------
    float : VaR (positive value represents potential loss)
    """
    if method == 'historical':
        # Historical method: use empirical quantile
        var = -np.percentile(returns, (1 - confidence_level) * 100)
        
    elif method == 'parametric':
        # Parametric method: assume normal distribution
        mu = returns.mean()
        sigma = returns.std()
        z_score = norm.ppf(1 - confidence_level)
        var = -(mu + sigma * z_score)
        
    elif method == 'cornish-fisher':
        # Cornish-Fisher: adjust for skewness and kurtosis
        mu = returns.mean()
        sigma = returns.std()
        skew = returns.skew()
        kurt = returns.kurtosis()
        
        z = norm.ppf(1 - confidence_level)
        z_cf = (z + 
                (z**2 - 1) * skew / 6 + 
                (z**3 - 3*z) * kurt / 24 - 
                (2*z**3 - 5*z) * skew**2 / 36)
        
        var = -(mu + sigma * z_cf)
    else:
        raise ValueError("method must be 'historical', 'parametric', or 'cornish-fisher'")
    
    return var


def calculate_cvar(returns, confidence_level=0.95):
    """
    Calculate Conditional VaR (CVaR) / Expected Shortfall.
    
    CVaR is the expected loss given that we are in the tail beyond VaR.
    """
    var = -np.percentile(returns, (1 - confidence_level) * 100)
    # Calculate mean of losses beyond VaR
    tail_losses = returns[returns <= -var]
    cvar = -tail_losses.mean() if len(tail_losses) > 0 else 0.0
    return cvar


# Calculate VaR and CVaR for AAPL
aapl_returns = returns['AAPL']
confidence_levels = [0.90, 0.95, 0.99]

print("Apple (AAPL) Risk Metrics:\n")
print("="*60)

for conf in confidence_levels:
    var_hist = calculate_var(aapl_returns, conf, 'historical')
    var_param = calculate_var(aapl_returns, conf, 'parametric')
    var_cf = calculate_var(aapl_returns, conf, 'cornish-fisher')
    cvar = calculate_cvar(aapl_returns, conf)
    
    print(f"\n{conf*100}% Confidence Level:")
    print(f"  Historical VaR:      {var_hist:.4%}")
    print(f"  Parametric VaR:      {var_param:.4%}")
    print(f"  Cornish-Fisher VaR:  {var_cf:.4%}")
    print(f"  CVaR (ES):           {cvar:.4%}")

print("\n" + "="*60)

In [ ]:
# Visualize VaR
fig, ax = plt.subplots(figsize=(14, 6))

# Plot histogram of returns
ax.hist(aapl_returns, bins=100, density=True, alpha=0.6, color='skyblue', edgecolor='black')

# Overlay normal distribution
mu, sigma = aapl_returns.mean(), aapl_returns.std()
x = np.linspace(aapl_returns.min(), aapl_returns.max(), 100)
ax.plot(x, norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal Distribution')

# Mark VaR levels
var_95_hist = -np.percentile(aapl_returns, 5)
var_95_param = -(mu + sigma * norm.ppf(0.05))
cvar_95 = calculate_cvar(aapl_returns, 0.95)

ax.axvline(-var_95_hist, color='red', linestyle='--', linewidth=2, label=f'95% VaR (Historical): {var_95_hist:.2%}')
ax.axvline(-cvar_95, color='darkred', linestyle='-.', linewidth=2, label=f'95% CVaR: {cvar_95:.2%}')

ax.set_xlabel('Daily Returns', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title('Apple (AAPL) Returns Distribution with VaR and CVaR', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Risk-Adjusted Performance Metrics

Raw returns don't tell the full story. We need to adjust for risk.

In [ ]:
def calculate_sharpe_ratio(returns, risk_free_rate=0.02, periods_per_year=252):
    """
    Calculate annualized Sharpe ratio.
    
    Sharpe Ratio = (Return - Risk_Free_Rate) / Volatility
    """
    excess_returns = returns - risk_free_rate / periods_per_year
    return np.sqrt(periods_per_year) * excess_returns.mean() / excess_returns.std()


def calculate_sortino_ratio(returns, risk_free_rate=0.02, periods_per_year=252):
    """
    Calculate annualized Sortino ratio (uses downside deviation).
    
    Sortino focuses only on downside volatility, which is more relevant for risk-averse investors.
    """
    excess_returns = returns - risk_free_rate / periods_per_year
    downside_returns = excess_returns[excess_returns < 0]
    downside_std = np.sqrt(np.mean(downside_returns**2))
    
    if downside_std == 0:
        return 0.0
    
    return np.sqrt(periods_per_year) * excess_returns.mean() / downside_std


def calculate_max_drawdown(returns):
    """
    Calculate maximum drawdown.
    
    Maximum drawdown is the largest peak-to-trough decline in cumulative returns.
    """
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return abs(drawdown.min())


# Calculate metrics for all stocks
metrics_df = pd.DataFrame(index=tickers)

for ticker in tickers:
    stock_returns = returns[ticker]
    
    metrics_df.loc[ticker, 'Ann. Return'] = stock_returns.mean() * 252
    metrics_df.loc[ticker, 'Ann. Volatility'] = stock_returns.std() * np.sqrt(252)
    metrics_df.loc[ticker, 'Sharpe Ratio'] = calculate_sharpe_ratio(stock_returns)
    metrics_df.loc[ticker, 'Sortino Ratio'] = calculate_sortino_ratio(stock_returns)
    metrics_df.loc[ticker, 'Max Drawdown'] = calculate_max_drawdown(stock_returns)
    metrics_df.loc[ticker, '95% VaR'] = calculate_var(stock_returns, 0.95, 'historical')
    metrics_df.loc[ticker, '95% CVaR'] = calculate_cvar(stock_returns, 0.95)

print("\nRisk-Adjusted Performance Metrics:\n")
print(metrics_df.round(4))

In [ ]:
# Visualize risk-return tradeoff
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Risk-Return scatter
ax1.scatter(metrics_df['Ann. Volatility'], metrics_df['Ann. Return'], s=200, alpha=0.6)
for ticker in tickers:
    ax1.annotate(ticker, 
                (metrics_df.loc[ticker, 'Ann. Volatility'], 
                 metrics_df.loc[ticker, 'Ann. Return']),
                fontsize=12, fontweight='bold')
ax1.set_xlabel('Annualized Volatility', fontsize=12)
ax1.set_ylabel('Annualized Return', fontsize=12)
ax1.set_title('Risk-Return Profile', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Sharpe ratios comparison
metrics_df['Sharpe Ratio'].sort_values().plot(kind='barh', ax=ax2, color='steelblue')
ax2.set_xlabel('Sharpe Ratio', fontsize=12)
ax2.set_title('Sharpe Ratio Comparison', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 4. Monte Carlo Simulation for Option Pricing

Monte Carlo simulation is a powerful technique for pricing complex derivatives.

In [ ]:
def monte_carlo_european_option(S0, K, T, r, sigma, option_type='call', n_simulations=100000):
    """
    Price European option using Monte Carlo simulation.
    
    Parameters:
    -----------
    S0 : float
        Current stock price
    K : float
        Strike price
    T : float
        Time to maturity (years)
    r : float
        Risk-free rate
    sigma : float
        Volatility
    option_type : str
        'call' or 'put'
    n_simulations : int
        Number of Monte Carlo paths
    
    Returns:
    --------
    tuple : (option_price, standard_error, simulated_prices)
    """
    # Simulate stock price at maturity using Geometric Brownian Motion
    # S_T = S_0 * exp((r - 0.5*sigma^2)*T + sigma*sqrt(T)*Z)
    Z = np.random.standard_normal(n_simulations)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Calculate payoffs
    if option_type == 'call':
        payoffs = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - ST, 0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    # Discount to present value
    option_price = np.exp(-r * T) * np.mean(payoffs)
    standard_error = np.exp(-r * T) * np.std(payoffs) / np.sqrt(n_simulations)
    
    return option_price, standard_error, ST


# Example: Price a call option on Apple
# Current Apple price
S0 = prices['AAPL'].iloc[-1]
K = S0  # At-the-money
T = 1.0  # 1 year
r = 0.05  # 5% risk-free rate
sigma = aapl_returns.std() * np.sqrt(252)  # Annualized volatility

print("\nMonte Carlo Option Pricing for Apple (AAPL)")
print("="*60)
print(f"Current Stock Price (S0): ${S0:.2f}")
print(f"Strike Price (K):         ${K:.2f}")
print(f"Time to Maturity (T):     {T} year")
print(f"Risk-free Rate (r):       {r:.2%}")
print(f"Volatility (σ):           {sigma:.2%}")
print("="*60)

# Price call option
call_price, call_se, ST_call = monte_carlo_european_option(S0, K, T, r, sigma, 'call', 100000)
print(f"\nCall Option Price:  ${call_price:.4f} ± ${call_se:.4f}")

# Price put option
put_price, put_se, ST_put = monte_carlo_european_option(S0, K, T, r, sigma, 'put', 100000)
print(f"Put Option Price:   ${put_price:.4f} ± ${put_se:.4f}")

# Verify Put-Call Parity: C - P = S0*e^(-q*T) - K*e^(-r*T)
# For no dividends (q=0): C - P = S0 - K*e^(-r*T)
parity_lhs = call_price - put_price
parity_rhs = S0 - K * np.exp(-r * T)
print(f"\nPut-Call Parity Check:")
print(f"  C - P = ${parity_lhs:.4f}")
print(f"  S - K*e^(-rT) = ${parity_rhs:.4f}")
print(f"  Difference: ${abs(parity_lhs - parity_rhs):.4f}")

In [ ]:
# Visualize Monte Carlo simulations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Distribution of terminal stock prices
ax1.hist(ST_call, bins=100, density=True, alpha=0.6, color='lightblue', edgecolor='black')
ax1.axvline(K, color='red', linestyle='--', linewidth=2, label=f'Strike = ${K:.2f}')
ax1.axvline(S0, color='green', linestyle='--', linewidth=2, label=f'Current = ${S0:.2f}')
ax1.set_xlabel('Stock Price at Maturity', fontsize=12)
ax1.set_ylabel('Density', fontsize=12)
ax1.set_title('Simulated Terminal Stock Prices', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Call option payoff distribution
call_payoffs = np.maximum(ST_call - K, 0)
ax2.hist(call_payoffs, bins=100, density=True, alpha=0.6, color='lightgreen', edgecolor='black')
ax2.axvline(call_price * np.exp(r * T), color='red', linestyle='--', linewidth=2, 
           label=f'Expected Payoff = ${call_price * np.exp(r * T):.2f}')
ax2.set_xlabel('Call Option Payoff', fontsize=12)
ax2.set_ylabel('Density', fontsize=12)
ax2.set_title('Call Option Payoff Distribution', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Heavy-Tailed Distributions in Finance

Financial returns often exhibit heavier tails than the normal distribution, leading to underestimation of extreme events.

In [ ]:
# Fit different distributions to AAPL returns
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Normal distribution
ax = axes[0, 0]
ax.hist(aapl_returns, bins=100, density=True, alpha=0.6, color='skyblue', edgecolor='black', label='Data')
mu, sigma = aapl_returns.mean(), aapl_returns.std()
x = np.linspace(aapl_returns.min(), aapl_returns.max(), 1000)
ax.plot(x, norm.pdf(x, mu, sigma), 'r-', linewidth=2, label=f'Normal(μ={mu:.4f}, σ={sigma:.4f})')
ax.set_title('Normal Distribution Fit', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Student-t distribution (heavier tails)
ax = axes[0, 1]
ax.hist(aapl_returns, bins=100, density=True, alpha=0.6, color='lightgreen', edgecolor='black', label='Data')
# Fit Student-t
params = student_t.fit(aapl_returns)
df, loc, scale = params
ax.plot(x, student_t.pdf(x, df, loc, scale), 'g-', linewidth=2, 
       label=f'Student-t(df={df:.2f})')
ax.plot(x, norm.pdf(x, mu, sigma), 'r--', linewidth=1, alpha=0.5, label='Normal (comparison)')
ax.set_title('Student-t Distribution Fit (Heavy Tails)', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Q-Q Plot vs Normal
ax = axes[1, 0]
stats.probplot(aapl_returns, dist="norm", plot=ax)
ax.set_title('Q-Q Plot vs Normal Distribution', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# 4. Statistics comparison
ax = axes[1, 1]
stats_text = f"""
AAPL Returns Statistics:

Mean:              {aapl_returns.mean():.6f}
Std Dev:           {aapl_returns.std():.6f}
Skewness:          {aapl_returns.skew():.4f}
Kurtosis:          {aapl_returns.kurtosis():.4f}

Normal Distribution:
  Skewness = 0
  Kurtosis = 0 (excess)

Interpretation:
{'Negative' if aapl_returns.skew() < 0 else 'Positive'} skew = {'Left' if aapl_returns.skew() < 0 else 'Right'} tail
Kurtosis > 0 = Heavier tails
            (more extreme events)
"""
ax.text(0.1, 0.5, stats_text, transform=ax.transAxes, fontsize=11, 
       verticalalignment='center', fontfamily='monospace',
       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax.axis('off')

plt.tight_layout()
plt.show()

print("\nKey Observations:")
print("1. Real returns have heavier tails than normal distribution")
print("2. Student-t distribution provides better fit")
print("3. Extreme events occur more frequently than normal model predicts")
print("4. This has important implications for risk management")

## 6. Portfolio Risk Analysis

Let's analyze a simple portfolio consisting of multiple stocks.

In [ ]:
# Create equal-weighted portfolio
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])  # Equal weights
portfolio_returns = (returns[tickers] * weights).sum(axis=1)

# Calculate portfolio metrics
print("\nPortfolio Analysis (Equal-Weighted)")
print("="*60)
print(f"\nWeights: {dict(zip(tickers, weights))}")
print(f"\nAnnualized Return:     {portfolio_returns.mean() * 252:.2%}")
print(f"Annualized Volatility: {portfolio_returns.std() * np.sqrt(252):.2%}")
print(f"Sharpe Ratio:          {calculate_sharpe_ratio(portfolio_returns):.4f}")
print(f"Maximum Drawdown:      {calculate_max_drawdown(portfolio_returns):.2%}")
print(f"95% VaR:               {calculate_var(portfolio_returns, 0.95, 'historical'):.2%}")
print(f"95% CVaR:              {calculate_cvar(portfolio_returns, 0.95):.2%}")

# Compare individual vs portfolio risk
print("\n" + "="*60)
print("Diversification Benefit:")
print("="*60)
avg_individual_vol = returns[tickers].std().mean() * np.sqrt(252)
portfolio_vol = portfolio_returns.std() * np.sqrt(252)
diversification_benefit = (avg_individual_vol - portfolio_vol) / avg_individual_vol
print(f"Average Individual Volatility: {avg_individual_vol:.2%}")
print(f"Portfolio Volatility:          {portfolio_vol:.2%}")
print(f"Risk Reduction:                {diversification_benefit:.2%}")

In [ ]:
# Visualize portfolio performance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Cumulative returns
ax = axes[0, 0]
cumulative_portfolio = (1 + portfolio_returns).cumprod()
cumulative_individual = (1 + returns[tickers]).cumprod()
cumulative_individual.plot(ax=ax, alpha=0.5, linewidth=1)
cumulative_portfolio.plot(ax=ax, color='black', linewidth=3, label='Portfolio')
ax.set_title('Cumulative Returns: Portfolio vs Individual Stocks', fontsize=12, fontweight='bold')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

# 2. Drawdown
ax = axes[0, 1]
running_max = cumulative_portfolio.expanding().max()
drawdown = (cumulative_portfolio - running_max) / running_max
drawdown.plot(ax=ax, color='red', linewidth=2, label='Portfolio Drawdown')
ax.fill_between(drawdown.index, drawdown.values, 0, alpha=0.3, color='red')
ax.set_title('Portfolio Drawdown Over Time', fontsize=12, fontweight='bold')
ax.set_ylabel('Drawdown')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Returns distribution
ax = axes[1, 0]
ax.hist(portfolio_returns, bins=100, density=True, alpha=0.6, color='steelblue', edgecolor='black')
mu, sigma = portfolio_returns.mean(), portfolio_returns.std()
x = np.linspace(portfolio_returns.min(), portfolio_returns.max(), 1000)
ax.plot(x, norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal Fit')
var_95 = calculate_var(portfolio_returns, 0.95, 'historical')
ax.axvline(-var_95, color='red', linestyle='--', linewidth=2, label=f'95% VaR: {var_95:.2%}')
ax.set_title('Portfolio Returns Distribution', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Correlation matrix
ax = axes[1, 1]
corr_matrix = returns[tickers].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
           square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Correlation Matrix', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## Key Takeaways

1. **VaR and CVaR**: Essential risk metrics, but parametric VaR can underestimate risk during market stress

2. **Risk-Adjusted Metrics**: Sharpe and Sortino ratios help compare investments on a risk-adjusted basis

3. **Monte Carlo**: Powerful for pricing complex derivatives and analyzing portfolio scenarios

4. **Heavy Tails**: Financial returns have fatter tails than normal distribution → extreme events are more likely

5. **Diversification**: Portfolio risk < sum of individual risks due to imperfect correlations

6. **Real Data**: Always validate models with real market data, not just theoretical distributions

## Exercises

1. Calculate VaR for different confidence levels (90%, 95%, 99%) and compare
2. Implement variance reduction techniques for Monte Carlo (antithetic variates, control variates)
3. Build a minimum variance portfolio using optimization
4. Backtest a VaR model using historical data
5. Implement Extreme Value Theory (EVT) for tail risk estimation